In [9]:
import pandas as pd
import numpy as np

from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

In [10]:
path = "../Data/To_analyze/Results_software/SIDER2_chr32/RUNS/run_2.csv"
data = pd.read_csv(path, sep=",", header=None)

In [11]:
data = data.dropna(axis=1, how='all')
data.columns = range(data.columns.size)
data.head()


,0,1,2,3,4,5
0,LinJ.01,666,24093,24758,plus,GGGGGAGGCGGGGGAGGCGGGGGGCACGCACCTCCATGCGTGGCAT...
1,LinJ.01,618,35316,35933,plus,GAGGTGGAGGCCGCTCTGCCCCCCCCGCCGCCGAGTGCTGCAGGCA...
2,LinJ.01,793,39789,40581,plus,TATTCTACCGCGAGCAAGGCAGCACACAGACGCACGCACAGCCACA...
3,LinJ.01,645,54885,55529,plus,TGTTTGGTCTTCCGCGTGTCCGTTTTCGCTGCCGCACACTGCGAGG...
4,LinJ.01,998,72760,73757,plus,CGTGTGTTTGTGAGGGGTAAGAATCGACCATGCATCAACGACTGCG...


In [12]:
def fasta_creator(data_df, fasta_output_path, selector):

    # data_df = pd.read_csv(path_input, sep=",", header=None)

    fasta_df = []
    for index, row in enumerate(data_df.iterrows(), start=1):
        if index in selector:
            row = row[1]
            rec = SeqRecord(
                Seq(row.iloc[5]),
                id="Seq_" + str(index) + "_" + row.iloc[0] + "_" + row.iloc[4],  
                description="Leishmania infantum " + row.iloc[4]
            )
            fasta_df.append(rec)
        else: continue

    SeqIO.write(fasta_df, fasta_output_path, "fasta")
    print("\nFasta created at:", fasta_output_path)

In [13]:
%%script false --no-raise-error
fasta_creator(data, "./first_seq.fasta", [1])

In [14]:
import os
def blastn_dic(path_input):
    # Remember is "path.input.dic_path" for "argparse".
    try:
        # boxymcboxface("BLASTn Database creator started")
        # pdb.set_trace()
        os.system("makeblastdb -in " + path_input + " -dbtype nucl -parse_seqids")
        print("\nBlast Dictionary created in", path_input)
    except Exception:
        print("\nError: Blast Dictionary couldn't be created")

In [15]:
%%script false --no-raise-error
blastn_dic("../Data/genome/TriTrypDB-67_LinfantumJPCM5_Genome.fasta")

In [16]:
import os
def blastn_blaster(query_path, dict_path, outfile_path, perc_identity):
    print("BLASTn searcher initiated")
    # pdb.set_trace()
    os.system("blastn -word_size 11 -query "
                + query_path + " -db "
                + dict_path + " -out "
                + outfile_path + " -perc_identity "
                + str(perc_identity) + str(0)
                + " -evalue 1.0E-09 "
                + " -outfmt '10 qseqid sseqid pident length qlen slen mismatch gapopen qstart qend sstart send evalue bitscore sstrand sseq'")
    print("\nBlaster successful", outfile_path, "created.")


In [17]:
%%script false --no-raise-error
blastn_blaster("./first_seq.fasta",
               "..//Data/genome/TriTrypDB-67_LinfantumJPCM5_Genome.fasta",
               "./test_1.csv",
               0)

In [25]:
import os
import subprocess

def blastn_blaster(query_path, dict_path, perc_identity):
    data = subprocess.check_output("blastn -word_size 11 -query " 
                + query_path + " -db " 
                + dict_path 
                + " -perc_identity " + str(perc_identity)
                + " -evalue 1.0E-09 -outfmt 10", shell=True, universal_newlines=True)
    return data

In [52]:
import subprocess
test = blastn_blaster("./first_seq.fasta",
               "..//Data/genome/TriTrypDB-67_LinfantumJPCM5_Genome.fasta",
               0)
print(test.count("\n"))  # Each row will have a "\n" at the end.

208
